In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# About this dataset
Age : Age of the patient

Sex : Sex of the patient

exang: exercise induced angina (1 = yes; 0 = no)

ca: number of major vessels (0-3)

cp : Chest Pain type chest pain type

Value 1: typical angina
Value 2: atypical angina
Value 3: non-anginal pain
Value 4: asymptomatic
trtbps : resting blood pressure (in mm Hg)

chol : cholestoral in mg/dl fetched via BMI sensor

fbs : (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)

rest_ecg : resting electrocardiographic results

Value 0: normal
Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
thalach : maximum heart rate achieved

target : 0= less chance of heart attack 1= more chance of heart attack

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics  import accuracy_score,classification_report,roc_auc_score,plot_roc_curve,plot_precision_recall_curve

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
data

# EXPLORATORY DATA ANALYSIS

In [ ]:
data.columns

In [ ]:
data.isnull().sum()#no missing value

In [ ]:
cat_var=['sex', 'cp','fbs', 'restecg', 'exng','slp', 'caa', 'thall' ]
continuous_var=['age','chol','trtbps','thalachh','oldpeak',]

## UNIVARIATE ANALYSIS

In [ ]:
for i in continuous_var:
    ax = sns.boxplot(data[i])
    plt.show()

### as we can see that the data has some outliers but we will not remove it ,because we might loss some important data and it is a medical data

### VISUALISATION

In [ ]:
sns.countplot(data=data,x='output')

#### no of more chance ofheart attacked case is higher than less chance of heart attack

In [ ]:

for i in cat_var:
    sns.countplot(data=data,x=data[i],hue='output')
    plt.show()

In [ ]:
for i in continuous_var:
    plt.figure(figsize=(10,8))
    sns.histplot(data=data,x=data[i],hue='output',kde=True)
    plt.show()
    

In [ ]:
#1-person having age 30-50 and 70+ has more chance of getting heart attack
#2-person having cholestrol 200+ has high chance of getting heart attack


In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(data.corr(),annot=True)

In [ ]:
corr_output=data.corr()['output'].sort_values(ascending=False)
corr_output

In [ ]:
plt.figure(figsize=(8,8))
corr_output.plot(kind='bar',color='green')

### here we can see  how all varaible related to the output(+ correlation,-correlation)

In [ ]:
sc=StandardScaler()
scaled_cont=sc.fit_transform(data[continuous_var])
scaled_cont=pd.DataFrame(scaled_cont,columns=continuous_var)
scaled_data=pd.concat([scaled_cont,data[cat_var]],axis=1)

In [ ]:
X=scaled_data
Y=data['output']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=.30,random_state=0)

In [ ]:
logreg=Pipeline([('logistic',LogisticRegression())])
decision=Pipeline([('dt',DecisionTreeClassifier())])
randomforest=Pipeline([('rf',RandomForestClassifier())])
naivebayes=Pipeline([('nb',GaussianNB())])
knn=Pipeline([('knn',KNeighborsClassifier())])
gbc=Pipeline([('gbc',GradientBoostingClassifier())])
adaboost=Pipeline([('adaboost',AdaBoostClassifier())])
sgdclassifier=Pipeline([('SGDclassifier',SGDClassifier())])
svc=Pipeline([('svc',SVC())])
mlpclass=Pipeline([('mlpc',MLPClassifier())])

In [ ]:
mypipeline1=[logreg,decision,randomforest,naivebayes,knn,gbc,adaboost,sgdclassifier,svc,mlpclass]
            

In [ ]:
accuracy=0.0
classifier=0
pipeline=""
pipelinedict1={0:'logistic',1:'dt',2:'rf',3:'nb',4:'knn',5:'gbc',6:'adaboost',7:'SGDclassifier',
              8:'svc',9:'mlpc'}

In [ ]:
for i in mypipeline1:
    i.fit(x_train,y_train)
model_score=[]
model_name=[]
for i,j in enumerate(mypipeline1):
    print("{} test accuracy: {}".format(pipelinedict1[i],j.score(x_test,y_test)))
    model_name.append(pipelinedict1[i])
    model_score.append(j.score(x_test,y_test))

In [ ]:
plt.figure(figsize=(8,8))
plt.barh(model_name,model_score,color='red')


### as we can see that KNN has the highest accuracy among all classifier algorithms ,that is 83.51%

## TUNING THE PARAMETER TO GET BETTER ACCURACY

In [ ]:
knn=KNeighborsClassifier()

In [ ]:
test_score=[]
train_score=[]
for i in range(1,50):
    knn=KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train,y_train)
    train_score.append(knn.score(x_train,y_train))
    test_score.append(knn.score(x_test,y_test))

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(range(1,50),train_score,color='green')
plt.plot(range(1,50),test_score,color='red')
plt.show()

### at n_neibhour=6,we are getting highest accuracy

In [ ]:
knn1=KNeighborsClassifier(n_neighbors=6)
knn1.fit(x_train,y_train)
knn1.score(x_test,y_test)

### here we have'nt got any improvement

In [ ]:
knn2=KNeighborsClassifier(n_neighbors=6)
rand_state=[]
accuracy=[]
for i in range(1,150,1):
    x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=.30,random_state=i)
    knn2.fit(x_train,y_train)
    accuracy.append(knn2.score(x_test,y_test))
    rand_state.append(i)
    

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(rand_state,accuracy)

### here random_state in between 100-120 we,are getting highest accuracy

In [ ]:
for i in range(len(rand_state)):
    print(accuracy[i],rand_state[i])

### at i=107(random_state) we are getting accuracy of 90.10%

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=.30,random_state=107)
knn2.fit(x_train,y_train)   

In [ ]:
knn2.score(x_test,y_test)

In [ ]:
y_test_pred=knn2.predict(x_test)


In [ ]:
accuracy_score(y_test,y_test_pred)

### here we have got accuracy of our model to be 90.10%

In [ ]:

plot_roc_curve(knn2,x_test,y_test)

### here we have got AUC SCORE=.93 which indicate that  model is too good.

In [ ]:
roc_auc_score(y_test,y_test_pred)

### ROC AUC SCORE =.90

In [ ]:
plot_precision_recall_curve(knn2,x_test,y_test)

### AVERAGE PRECISION SCORE IS-- AP=.93

In [ ]:
print(classification_report(y_test,y_test_pred))